# 🏥 Adaptive Reasoning for Healthcare Administration

## Real-World Multi-Task Reasoning with Symbolic AI

This demo showcases an **adaptive LLM-symbolic reasoning framework** solving realistic healthcare administration tasks, automatically routing each problem to the optimal solver.

---

### 📋 Scenario: Hospital Admin Assistant

A hospital administrator faces three different reasoning tasks in a single workflow:

<div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px; margin-top: 20px;">

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h4 style="margin-top: 0; color: white;">📊 Task 1: Clinical Trial Matching</h4>
<p style="font-size: 0.9em; margin-bottom: 5px;"><strong>Type:</strong> Constraint Satisfaction</p>
<p style="font-size: 0.85em; opacity: 0.95;">Match patient profile against complex trial inclusion/exclusion criteria</p>
<p style="font-size: 0.8em; margin-top: 10px; opacity: 0.9;">⚙️ Optimal solver: Z3 (SMT)</p>
</div>

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 20px; border-radius: 10px; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h4 style="margin-top: 0; color: white;">🔐 Task 2: Policy Compliance</h4>
<p style="font-size: 0.9em; margin-bottom: 5px;"><strong>Type:</strong> First-Order Logic</p>
<p style="font-size: 0.85em; opacity: 0.95;">Verify staff credentialing against hospital policies</p>
<p style="font-size: 0.8em; margin-top: 10px; opacity: 0.9;">⚙️ Optimal solver: Prover9</p>
</div>

<div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); padding: 20px; border-radius: 10px; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h4 style="margin-top: 0; color: white;">✅ Task 3: Discharge Readiness</h4>
<p style="font-size: 0.9em; margin-bottom: 5px;"><strong>Type:</strong> Logic Programming</p>
<p style="font-size: 0.85em; opacity: 0.95;">Evaluate patient discharge eligibility via forward chaining</p>
<p style="font-size: 0.8em; margin-top: 10px; opacity: 0.9;">⚙️ Optimal solver: Pyke</p>
</div>

</div>

---

### 🎯 Framework Capabilities

✨ **Automatic Problem Detection** - Identifies reasoning type without manual specification  
🧠 **Heterogeneous Solver Selection** - Routes to optimal symbolic solver (Z3, Prover9, Pyke)  
⚡ **Parallel Execution** - Processes independent tasks concurrently  
🔄 **End-to-End Automation** - From natural language to verified answers


## Step 1: Load Hospital Admin Tasks

Loading three realistic healthcare administration tasks that would typically be handled by different software systems.


In [1]:
problem_description = '''
PATIENT CASE FOR TRIAL MATCHING:
Patient Profile:
- 62-year-old male
- History: CVA, neurogenic bladder with indwelling suprapubic catheter
- Multiple prior admissions for UTIs, altered mental status, and urosepsis
- Current: Urosepsis (now resolved after vanc/meropenem treatment)
- CT findings: Non-obstructing stone in left ureter, no urethral strictures
- Medical history: Non-Hodgkins Marginal Zone Lymphoma (s/p R-CHOP x6), Bell's Palsy, BPH, Hypertension, Partial bowel obstruction s/p colostomy, Hepatitis C, Cryoglobulinemia, SLE with transverse myelitis (anti-dsDNA Ab+), Insulin-dependent diabetes, Fungal esophagitis, Recurrent UTIs (Pseudomonas & Enterococcus)

Clinical Trial: NCT02142751
Trial Criteria:
Inclusion:
- ≥18 years old hospitalized patients
- Negative pregnancy test in fertile women
- Episode of monomicrobial urinary BSI due to multidrug-resistant E.coli (susceptible to fosfomycin and meropenem or ceftriaxone)
- Urinary sepsis with MDR E.coli from blood cultures, requires:
  Clinical criteria (at least one): UTI symptoms (dysuria, urgency, suprapubic pain, pollakiuria), lumbar back pain, costovertebral angle tenderness, altered mental status (≤70 years), intermittent/permanent indwelling Foley catheter
  Urinalysis criteria (at least one): Positive dipstick for nitrites or leukocyte esterase, positive urine culture

Exclusion criteria:
- Polymicrobial bacteremia
- Undrained renal abscess or unresolved obstructive uropathy
- Pregnant or breastfeeding women
- Hematogenous infection
- Other concomitant infection
- Renal transplant recipients
- Polycystic kidney
- Hypersensitivity to meropenem, fosfomycin, or ceftriaxone

Based on the available information, can this patient be enrolled in the trial?
A) Yes
B) No

HOSPITAL CREDENTIALING VERIFICATION:
Excerpt from Hospital Policy Manual Section - Staff Credentialing Requirements:
- All hospital staff members shall be classified as either clinical staff or administrative staff.
- All personnel who provide direct patient care services are classified as clinical staff.
- No staff members classified as administrative staff provide direct patient care services.
- All staff who are classified as clinical staff must complete annual infection control training to receive credentialing authorization.
- All staff who are classified as clinical staff must complete annual HIPAA certification to receive credentialing authorization.
- No billing clerks are classified as clinical staff.
- All registered nurses provide direct patient care services.
- No staff who have not completed annual infection control training are credentialed.
- No staff who have not completed annual HIPAA certification are credentialed.

Personnel Record for Johnson, R.N.:
- Position: Registered Nurse
- Mandatory Compliance Status: Annual infection control training completed; Annual HIPAA certification not completed

Question: According to the system, Nurse Johnson is not credentialed. Is this correct?
A) Yes
B) No

DISCHARGE READINESS VERIFICATION:
Patient Discharge Protocol - Standard Operating Procedure:
- If a patient has stable vital signs, then the patient meets vital stability criteria.
- If a patient is off IV antibiotics, then the patient meets antibiotic transition criteria.
- If a patient meets vital stability criteria and meets antibiotic transition criteria, then the patient is medically stable.
- If a patient can perform self-care activities, then the patient is functionally independent.
- If a patient is functionally independent and has completed discharge teaching, then the patient meets discharge education requirements.
- If a patient is medically stable and meets discharge education requirements, then the patient is cleared for discharge.

Patient Status Report - Chen, Margaret (MRN: 445821):
Margaret Chen has stable vital signs. Margaret Chen is off IV antibiotics. Margaret Chen can perform self-care activities. Margaret Chen has completed discharge teaching.

Based on this discharge protocol, is Margaret Chen cleared for discharge?
A) Yes
B) No
'''

## Step 2: Devise Plan for Adaptive Reasoning 

The **Carnap Router** analyzes tasks and creates an execution plan.

Let's first initialize load the config and initialize an LLM:


In [2]:
import yaml
from agents.generation.api import AzureOpenAIGenerator
from agents.meta_agents.planner import Planner, TracePersister
from helpers.trace_explainer import build_trace_payload, why
# Load config and initialize
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
azure_config = config['api_config']['gpt-4o-azure']
llm = AzureOpenAIGenerator(
    model_name=azure_config['model_name'],
    api_key=azure_config['api_key'],
    model_version=azure_config['openai_api_version'],
    azure_endpoint=azure_config['azure_endpoint']
)


/idiap/temp/lxu/miniconda3/envs/carnap_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now, the router:

- Parses the multi-task input
- Classifies each task by reasoning type
- Constructs an execution DAG
- Routes tasks to appropriate symbolic solvers

In [3]:
router = Planner(generator=llm)
plans, memory, problem_ids = router({"problem": problem_description})
plan = plans[0]


2025-11-07 19:41:45,812 | INFO | Registered agent '<PLAN_START>'
2025-11-07 19:41:45,813 | INFO | Registered agent '<PLAN_END>'
2025-11-07 19:41:45,813 | INFO | Registered agent 'lp_solver'
2025-11-07 19:41:45,814 | INFO | Registered agent 'fol_solver'
2025-11-07 19:41:45,814 | INFO | Registered agent 'csp_solver'
2025-11-07 19:41:45,814 | INFO | Registered agent 'smt_solver'
2025-11-07 19:41:45,815 | INFO | Registered agent 'ilp_solver'
2025-11-07 19:41:45,815 | INFO | Registered agent 'epistemic_solver'
2025-11-07 19:41:45,816 | INFO | Registered agent 'risk_solver'
2025-11-07 19:41:45,816 | INFO | Registered agent 'compositional_solver'
2025-11-07 19:41:45,816 | INFO | Registered agent 'causal_solver'
/remote/idiap.svm/temp.rea01/lxu/251009_Carnap_clean/workshop_release/adaptive_symbolic_reasoning/agents/generation/api.py:203: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instea

## Step 3: Let CARNAP Solve It

We can now execute the plan!


In [4]:
trace_logger = TracePersister()
plan.execute(memory, trace_logger)


2025-11-07 19:41:57,712 | INFO | Execute plan with topological order...
2025-11-07 19:41:57,713 | INFO | Trace saved: <PLAN_START> → None
2025-11-07 19:41:57,714 | INFO | Trace saved: ques_3 → None
2025-11-07 19:41:57,714 | INFO | Trace saved: ques_2 → None
2025-11-07 19:41:57,714 | INFO | Trace saved: ques_1 → None
2025-11-07 19:42:01,768 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-11-07 19:42:01,772 | INFO | Scratchpad WRITE intermediate_form_lp_solver=Predicates:
stable_vitals($x, bool) ::: Does patient x have  (ttl=None)
2025-11-07 19:42:01,773 | INFO | Attempt 1/1
writing [compiled_krb]/facts.fbc
writing [compiled_krb]/rules_fc.py
writing [compiled_krb]/compiled_pyke_files.py
2025-11-07 19:42:01,823 | INFO | Scratchpad WRITE critique_outputs_lp_solver=True (ttl=None)
2025-11-07 19:42:01,823 | INFO | Pyke solver ran successfully.
2025-11-07 19:42:02,254 | INFO | 

Collecting results from memory:

In [5]:
predictions = []
for pid in problem_ids:
    result = memory.read(f"result_{pid}") or {}
    predictions.append(result.get('parsed_answer', None))

print(predictions)

['A', 'A', 'A']


The three predictions correspond to our three tasks, where **A = Yes** and **B = No**. For example, ['A', 'B', 'A']
 means:

1. **Clinical Trial Matching:** A → Patient **can** be enrolled
2. **Policy Compliance:** B → Nurse Johnson is **not** credentialed
3. **Discharge Readiness:** A → Margaret Chen is **cleared** for discharge

## Generate "Why" HTML
Package the execution trace and send it to the `why()` helper to refresh `demo_hospital_admin.html`.


In [6]:
import yaml
from agents.generation.api import AzureOpenAIGenerator
from agents.meta_agents.planner import Planner, TracePersister
from helpers.trace_explainer import build_trace_payload

result_summary = []
for pid in problem_ids:
    result = memory.read(f"result_{pid}") or {}
    result_summary.append({
        "problem_id": pid,
        "parsed_answer": result.get("parsed_answer"),
        "raw_solver_output": result.get("ori_answer"),
        "solver_name": result.get("solver_name"),
    })

trace_payload = build_trace_payload(
    plan,
    trace_logger,
    memory=memory,
    metadata={
        "scenario": "Hospital admin assistant",
        "problem_ids": problem_ids,
        "result_summary": result_summary,
    },
    problem_statement=problem_description,
)
print(f"Captured {len(trace_payload['trace'])} trace events for {len(problem_ids)} tasks.")


Captured 8 trace events for 3 tasks.


In [7]:
print(trace_payload)

{'plan': {'agents': [{'name': '<PLAN_START>', 'goal': 'Special control marker, indicating the start of the whole working plan.', 'predecessors': [], 'problem_to_solve': []}, {'name': 'ques_3', 'goal': 'Virtual agent representing ques_3', 'predecessors': ['<PLAN_START>'], 'problem_to_solve': ['ques_3']}, {'name': 'ques_2', 'goal': 'Virtual agent representing ques_2', 'predecessors': ['<PLAN_START>'], 'problem_to_solve': ['ques_2']}, {'name': 'ques_1', 'goal': 'Virtual agent representing ques_1', 'predecessors': ['<PLAN_START>'], 'problem_to_solve': ['ques_1']}, {'name': 'lp_solver', 'goal': 'Determine whether a hypothesis can be derived from the given facts and rules using logical deduction..', 'predecessors': ['ques_3'], 'problem_to_solve': ['ques_3']}, {'name': 'fol_solver', 'goal': 'A theorem prover specialized for problems expressed in first-order logic (FOL). It is capable of handling complex reasoning tasks that involve quantifiers, predicates, and relationships among multiple ent

In [8]:
import importlib
import helpers.trace_explainer
importlib.reload(helpers.trace_explainer)
from helpers.trace_explainer import why

from IPython.display import HTML

task_titles = {
    "ques_1": "Clinical Trial Enrollment",
    "ques_2": "Credential Compliance",
    "ques_3": "Discharge Clearance",
}
answer_summary = ", ".join(
    f"{task_titles.get(item['problem_id'], item['problem_id'])}: {item.get('parsed_answer')}"
    for item in result_summary
    if item.get('parsed_answer')
)
narrative_context = (
    "Carnap routed three hospital-admin subtasks (trial matching, credentialing, discharge) through the symbolic plan."
)
style_hint = (
    "Highlight the DAG, show per-task outcomes (A/B) as chips, and emphasise why each solver decision is trustworthy."
)
extra_guidance = (
    "Name each task explicitly using the provided task_titles mapping and echo the answer summary: " + (answer_summary or "pending answers.")
)
html = why(
    llm,
    trace_payload,
    narrative_context=narrative_context,
    style_hint=style_hint,
    extra_guidance=extra_guidance,
    output_path="demo_hospital_admin.html",
    model_args={"temperature": 0.15, "max_tokens": 6000},
)
# HTML(html)


2025-11-07 19:42:20,505 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 429 Too Many Requests"
2025-11-07 19:42:20,506 | INFO | Retrying request to /chat/completions in 38.000000 seconds
2025-11-07 19:43:06,900 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-11-07 19:43:07,101 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 429 Too Many Requests"
2025-11-07 19:43:07,102 | INFO | Retrying request to /chat/completions in 52.000000 seconds
2025-11-07 19:44:05,072 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-11-07 19:44:11,958 | INFO | HTTP Request: POS